In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import regex as re
from collections import Counter
import pickle
import json
import time
from scipy import sparse


In [ ]:
data = pd.read_csv('/kaggle/input/final-reduced-recipe-corpus-v1/final_reduced_recipe_corpus')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (111,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data = data.drop('Unnamed: 0', axis = 1)
data

,recp_id,0,1,2,3,4,5,6,7,8,...,134,135,136,137,138,139,140,141,142,143
0,137739,arriba,baked,winter,squash,mexican,style,60,minute,le,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31490,bit,different,breakfast,pizza,30,minute,le,time,make,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,112140,kitchen,chili,time,make,course,preparation,main,dish,chili,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59389,alouette,potato,60,minute,le,time,make,course,main,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44061,amish,tomato,ketchup,canning,weeknight,time,make,course,main,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231614,486161,zydeco,soup,ham,60,minute,le,time,make,course,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231615,493372,zydeco,spice,mix,15,minute,le,time,make,course,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231616,308080,zydeco,ya,ya,deviled,egg,60,minute,le,time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231617,298512,cooky,design,cooky,stick,30,minute,le,time,make,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
RAW_interactions = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv')
# /kaggle/input/food-com-recipes-and-user-interactions/ingr_map.pkl
PP_recipes = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/PP_recipes.csv')
RAW_recipes = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv')
interactions_train = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/interactions_train.csv')
interactions_test = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/interactions_test.csv')
PP_users = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/PP_users.csv')
interactions_validation = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/interactions_validation.csv')


In [ ]:
main_document = []

for i in range(data.shape[0]):
    test_str = ''
    
    for word in list(data.iloc[i])[1:]:
        if not isinstance(word, float):
            test_str += word + ' '
    
    main_document.append(test_str)

In [ ]:
len(PP_users.iloc[0]['ratings']), len(PP_users.iloc[0]['items'])

(155, 221)

In [ ]:
# main_document
main_document = np.array(main_document)

In [ ]:
tfidf = TfidfVectorizer()
result = tfidf.fit_transform(main_document)

In [ ]:
# sparse.save_npz("/kaggle/working/final_tfidf_mat.npz", result)
# your_matrix_back = sparse.load_npz("yourmatrix.npz")

In [ ]:
# test on users
PP_users.iloc[0]

u                                                             0
techniques    [8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...
items         [1118, 27680, 32541, 137353, 16428, 28815, 658...
n_items                                                      31
ratings       [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...
n_ratings                                                    31
Name: 0, dtype: object

In [ ]:
items_interacted = list(map(int,PP_users.iloc[0]['items'][1:-1].split(', ')))
# items_interacted

In [ ]:
# getting receipe id from items 
receipe_id = list(PP_recipes[PP_recipes['i'].isin(items_interacted)]['id'])
# receipe_id

In [ ]:
index_in_main_doc = list(data[data['recp_id'].isin(receipe_id)].index)
# index_in_main_doc

In [ ]:
user_str = ' '.join([main_document[i] for i in index_in_main_doc])
# user_str

In [ ]:
# converting the user_str to array based on our corpus and finding the cosine similarity.
# it will take some time as we have 2.3 lac recipes
new_user_arr = tfidf.transform([user_str]).toarray()

test = [cosine_similarity(new_user_arr, x.toarray()) for x in result]

In [ ]:
temp_test = np.array(test)[:,0,0]
# temp_test

In [ ]:
# recommending 5 recipes that user has no interacted with based on his/her taste
count = -1
i = 0
sorted_index = np.argsort(temp_test)
while i < 5:
    if not data.iloc[sorted_index[count]]['recp_id'] in receipe_id:
        print(data.iloc[sorted_index[count]]['recp_id'], list(RAW_recipes[RAW_recipes['id'] == data.iloc[sorted_index[count]]['recp_id']]['name'])[0])
        count -= 1
        i += 1
    else:
        count -= 1
    

83822 black bean onion soup
25306 quick and easy seasoned black beans
315425 delicious dirty rice
56201 turkey black bean chili
67639 green beans with tomatoes


In [ ]:
for r in receipe_id:
    print(r, list(RAW_recipes[RAW_recipes['id'] == r]['name'])[0])

44244 portuguese pork ragout with sweet peppers
40852 sauteed salmon with lime honey sauce
69501 fried broccoli
39907 beans  black eyed peas  and cashew salad
4764 2 step pumpkin cheesecake
36653 easy creamy yogurt pie
26259 mediterranean broccoli salad
16391 yummy n  easiest  warm blueberry sauce
40893 white bean   green chile pepper soup
49387 oven fried eggplant  aubergine
72929 snow peas with red bell peppers
63965 lentil and orzo salad
38104 rocky mount chicken
32377 basil  cilantro and macadamia nut chicken
81845 southwest spaghetti squash
20500 toasted orzo pilaf
16954 black beans with mango sauce
40753 warm spinach salad
30565 oh my d lux chocolate chip cookies
34509 bananas au gratin
40923 quick broccoli slaw
34513 roasted asparagus   shiitake mushrooms
79964 squash casserole  charlotte s barclay cafeteria
69545 low carb key lime pie
77818 veal cutlets with olive  tomato and anchovy sauce
18007 key lime cheesecake squares
36381 sweet potato stew with orange juice and black bea

### test for faster solution

In [ ]:
new_user_arr = tfidf.transform([user_str]).toarray()

cosine_similarity(new_user_arr, result[np.random.randint(0, result.shape[0] - 1, 10000)].toarray())

array([[0.27613114, 0.08729421, 0.15544289, ..., 0.28076402, 0.16479874,
        0.18851434]])

In [ ]:
def get_similarity_array(user_id, PP_users, main_document,doc_mat, data, RAW_recipes, n):
    items_interacted = list(map(int,PP_users[PP_users['u'] == user_id]['items'][user_id][1:-1].split(', ')))
    user_str = ' '.join(main_document[items_interacted])
    new_user_arr = tfidf.transform([user_str]).toarray()
    rnd_no = np.random.randint(0, doc_mat.shape[0] - 1, n)
    cs = cosine_similarity(new_user_arr, doc_mat[rnd_no].toarray())
#     cs = np.array([cosine_similarity(new_user_arr, x.toarray()) for x in doc_mat])
    items_interacted = list(map(int,PP_users.iloc[user_id]['items'][1:-1].split(', ')))
    receipe_id = list(PP_recipes[PP_recipes['i'].isin(items_interacted)]['id'])
    sorted_index = np.argsort(cs)[0]
    count = -1
    i = 0
    while i < 5:
        if not data.iloc[rnd_no[sorted_index[count]]]['recp_id'] in receipe_id:
            test = main_document[rnd_no[sorted_index[count]]]
            test = tfidf.transform([test]).toarray()
            print(data.iloc[rnd_no[sorted_index[count]]]['recp_id'], list(RAW_recipes[RAW_recipes['id'] == data.iloc[rnd_no[sorted_index[count]]]['recp_id']]['name'])[0], np.linalg.norm(test - new_user_arr))
            count -= 1
            i += 1
        else:
            count -= 1

In [ ]:
t = time.time()
get_similarity_array(1001,  PP_users, main_document, result, data, RAW_recipes, 10000)
print(f'Time taken -> {time.time() - t}')

16166 stuffed baked tomatoes 1.1104449872992224
406519 southwestern shepherds pie with chicken and chili mashed potato 1.1110570988821187
281845 sunday chicken stew 1.1127539020923625
27729 sunday dinner chicken 1.118673438456806
34653 witches  breath dip 1.118990976145168
Time taken -> 4.437320947647095


## Conversational RS

In [ ]:
user_id = 100
ep = []
n = 10000
items_interacted = list(map(int,PP_users[PP_users['u'] == user_id]['items'][user_id][1:-1].split(', ')))
selected = None
lmbda = 0.6
while True:
    if not selected:
        user_str = ' '.join(main_document[items_interacted])
        new_user_arr = tfidf.transform([user_str]).toarray()
        rnd_no = np.random.randint(0, result.shape[0] - 1, n)
        cs = cosine_similarity(new_user_arr, result[rnd_no].toarray())
        sorted_index = np.argsort(cs)[0]
        count = -1
        i = 0

        temp = []
        while i < 5:
            if not data.iloc[rnd_no[sorted_index[count]]]['recp_id'] in receipe_id:
                temp.append(data.iloc[rnd_no[sorted_index[count]]]['recp_id'])
                print(abs(i + 1), ':',data.iloc[rnd_no[sorted_index[count]]]['recp_id'], list(RAW_recipes[RAW_recipes['id'] == data.iloc[rnd_no[sorted_index[count]]]['recp_id']]['name'])[0])
                count -= 1
                i += 1
            else:
                count -= 1

        user_inp = input('Do you like any? Enter YES after the number of dish if you liked or enter the number of dish you would like to see similar.')

        if 'YES' in user_inp:
            print('Adding this dish to your profile.')
            print('Enjoy your dish!!')
            break
        else:
            print()
            selected = temp[int(user_inp)-1]
    else:
        ep.append(main_document[data[data['recp_id'] == selected].index[0]])
        ep_str = ' '.join(ep)
        ep_user_arr = tfidf.transform([ep_str]).toarray()
        
        final_user_arr = lmbda * new_user_arr + (1 - lmbda) * ep_user_arr
        
        rnd_no = np.random.randint(0, result.shape[0] - 1, n)
        cs = cosine_similarity(final_user_arr, result[rnd_no].toarray())
        sorted_index = np.argsort(cs)[0]
        count = -1
        i = 0

        temp = []
        while i < 5:
            if not data.iloc[rnd_no[sorted_index[count]]]['recp_id'] in receipe_id:
                temp.append(data.iloc[rnd_no[sorted_index[count]]]['recp_id'])
                print(abs(i + 1), ':',data.iloc[rnd_no[sorted_index[count]]]['recp_id'], list(RAW_recipes[RAW_recipes['id'] == data.iloc[rnd_no[sorted_index[count]]]['recp_id']]['name'])[0])
                count -= 1
                i += 1
            else:
                count -= 1

        user_inp = input('Do you like any? Enter YES after the number of dish if you liked or enter the number of dish you would like to see similar.')

        if 'YES' in user_inp:
            print('Adding this dish to your profile.')
            print('Enjoy your dish!!')
            break
        else:
            print()
            selected = temp[int(user_inp)-1]

1 : 128046 30 minute one pan chicken meal
2 : 153826 spicy pepper chicken pasta
3 : 227606 lemon and tomato chicken
4 : 187096 chicken pepper skillet
5 : 95472 caramelized tomatoes


Do you like any? Enter YES after the number of dish if you liked or enter the number of dish you would like to see similar. 4



1 : 191339 braised chicken with green peppers and tomatoes
2 : 147775 chicken zing
3 : 160982 herbed chicken tenders
4 : 67423 chicken   peppers
5 : 392831 chicken and rice chowder


Do you like any? Enter YES after the number of dish if you liked or enter the number of dish you would like to see similar. 3



1 : 80868 gingered chicken breast
2 : 227606 lemon and tomato chicken
3 : 367240 chicken calle ocho
4 : 383171 creamy chicken and spinach pasta
5 : 105598 shredded mexican chicken


##Refernces:
Datasets: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions